<h1 align="center">Hidden Markov Models on Weather Dataset<h1>

## First We import the libraries required for our project

In [25]:
import pandas as pd
import numpy as np

### Let's import the weather dataset for our project

In [26]:
file_path=".//Data.txt"
data=pd.read_csv((file_path),names=["weather","umbrella"])
data.head()

,weather,umbrella
0,foggy,no
1,foggy,no
2,foggy,no
3,rainy,yes
4,sunny,no


## Calculation of Transition and Emission probabilites:

In [27]:
groups=(data.groupby([data["weather"],data["umbrella"]=="yes"],as_index=False).count())
groups

,weather,umbrella
0,foggy,182
1,foggy,79
2,rainy,46
3,rainy,199
4,sunny,452
5,sunny,42


In [28]:
counts=dict()
key=""
for i in groups.iterrows():
    if key!=i[1].iloc[0]:
        key=i[1].iloc[0]
        counts[key]={}
        counts[key]["no"]=i[1].iloc[1]
    else:
        counts[key]["yes"]=i[1].iloc[1] 
counts

{'foggy': {'no': 182, 'yes': 79},
 'rainy': {'no': 46, 'yes': 199},
 'sunny': {'no': 452, 'yes': 42}}

In [29]:
probabilities=dict()
for i in counts.keys():
    probabilities[i]={}
    probabilities[i]["yes"]=counts[i]["yes"]/(counts[i]["yes"]+counts[i]["no"])
    probabilities[i]["no"]=counts[i]["no"]/(counts[i]["yes"]+counts[i]["no"])
    probabilities[i]["total"]=(counts[i]["yes"]+counts[i]["no"])/len(data)
probabilities

{'foggy': {'yes': 0.30268199233716475,
  'no': 0.6973180076628352,
  'total': 0.261},
 'rainy': {'yes': 0.8122448979591836,
  'no': 0.18775510204081633,
  'total': 0.245},
 'sunny': {'yes': 0.08502024291497975,
  'no': 0.9149797570850202,
  'total': 0.494}}

In [30]:
prob_dframe=pd.DataFrame(probabilities)
prob_dframe.drop(["total"],axis=0)
prob_dframe=prob_dframe.drop(["total"],axis=0)
print("Emission Probabilities:")
prob_dframe

Emission Probabilities:


,foggy,rainy,sunny
no,0.697318,0.187755,0.91498
yes,0.302682,0.812245,0.08502


### Now let's count the number of occurences of each transition in our dataset to calculate the transition probabilities

In [36]:
tran_cnt=dict()
tran_cnt["foggy"]={"rainy":0,"sunny":0,"foggy":0}
tran_cnt["rainy"]={"sunny":0,"foggy":0,"rainy":0}
tran_cnt["sunny"]={"rainy":0,"foggy":0,"sunny":0}
prev_state=""
occ={"foggy":0,"rainy":0,"sunny":0}
for i in data.iterrows():
    if prev_state=="":
        prev_state=i[1].iloc[0]
        occ[prev_state]+=1
    else:
        tran_cnt[prev_state][i[1].iloc[0]]+=1
        prev_state=i[1].iloc[0]
        occ[prev_state]+=1
occ[prev_state]-=1
print("Transtion counts: \n",tran_cnt)
print("Total Number of Occurence of each event: \n",occ)
tran_prob=dict()
tran_prob["foggy"]={"rainy":0,"sunny":0,"foggy":0}
tran_prob["rainy"]={"sunny":0,"foggy":0,"rainy":0}
tran_prob["sunny"]={"rainy":0,"foggy":0,"sunny":0}

#Calculating the probability of the transition

tran_prob["foggy"]["foggy"]=tran_cnt["foggy"]["foggy"]/occ["foggy"]###  Transition Probability for the transtion
tran_prob["foggy"]["rainy"]=tran_cnt["foggy"]["rainy"]/occ["foggy"]#    from foggy to any
tran_prob["foggy"]["sunny"]=tran_cnt["foggy"]["sunny"]/occ["foggy"]###  other state.

tran_prob["rainy"]["foggy"]=tran_cnt["rainy"]["foggy"]/occ["rainy"]###  Transition Probability for the transtion
tran_prob["rainy"]["rainy"]=tran_cnt["rainy"]["rainy"]/occ["rainy"]#    from rainy to any
tran_prob["rainy"]["sunny"]=tran_cnt["rainy"]["sunny"]/occ["rainy"]###  other state

tran_prob["sunny"]["foggy"]=tran_cnt["sunny"]["foggy"]/occ["sunny"]###  Transition Probability for the transtion
tran_prob["sunny"]["rainy"]=tran_cnt["sunny"]["rainy"]/occ["sunny"]#    form sunny to any
tran_prob["sunny"]["sunny"]=tran_cnt["sunny"]["sunny"]/occ["sunny"]###  other state

print("\n\nTransition Probabilities: ",tran_prob)

Transtion counts:  {'foggy': {'rainy': 76, 'sunny': 54, 'foggy': 131}, 'rainy': {'sunny': 48, 'foggy': 55, 'rainy': 142}, 'sunny': {'rainy': 27, 'foggy': 74, 'sunny': 392}}
Total Number of Occurence of each event:  {'foggy': 261, 'rainy': 245, 'sunny': 493}


Transition Probabilities:  {'foggy': {'rainy': 0.29118773946360155, 'sunny': 0.20689655172413793, 'foggy': 0.5019157088122606}, 'rainy': {'sunny': 0.19591836734693877, 'foggy': 0.22448979591836735, 'rainy': 0.5795918367346938}, 'sunny': {'rainy': 0.05476673427991886, 'foggy': 0.15010141987829614, 'sunny': 0.795131845841785}}


### Now let's load the transition probability into a dataframe for better use and visualization

In [37]:
tran_probFrame=pd.DataFrame(tran_prob)
tran_probFrame=tran_probFrame.T #We will be transposing the matrix just for displaying purpose but for actual calculation we
tran_probFrame                  #will still use the orginal dataframe

,foggy,rainy,sunny
foggy,0.501916,0.291188,0.206897
rainy,0.224490,0.579592,0.195918
sunny,0.150101,0.054767,0.795132


In [38]:
tran_probFrame=tran_probFrame.T #Restore to original format for actual calculation 

## Now we will calculate the alpha at each timestep i.e we will use the Forward Algorithm to calculate the sequence of hidden states

### Given Sequence of Observed Events

In [41]:
sequence=['no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'yes']

### Now let's initialize our intial probabilities and alpha matrix

In [42]:
alpha=[]

### Since our initial state is given as "sunny" our initial probabilities will be as follows:

In [43]:
state_probabilities={"foggy":0,"rainy":0,"sunny":1}

## Now Let's create our Hidden Markov Model:

### Forward Algorithm:

In [44]:
def forward_algorithm(state_probabilities,alpha):
    alpha.append(dict(state_probabilities))
    for i in sequence:
        curr_s,curr_r,curr_f=state_probabilities["sunny"],state_probabilities["rainy"],state_probabilities["foggy"]
        
        #Calcuating the probability for the next timestep given an observation
        nxt_s=((curr_s*tran_probFrame["sunny"]["sunny"])+(curr_r*tran_probFrame["rainy"]["sunny"])+(curr_f*tran_probFrame["foggy"]["sunny"]))*prob_dframe["sunny"][i]
        nxt_r=((curr_s*tran_probFrame["sunny"]["rainy"])+(curr_r*tran_probFrame["rainy"]["rainy"])+(curr_f*tran_probFrame["foggy"]["rainy"]))*prob_dframe["rainy"][i]
        nxt_f=((curr_s*tran_probFrame["sunny"]["foggy"])+(curr_r*tran_probFrame["rainy"]["foggy"])+(curr_f*tran_probFrame["foggy"]["foggy"]))*prob_dframe["foggy"][i]
        
        state_probabilities["sunny"],state_probabilities["rainy"],state_probabilities["foggy"]=nxt_s,nxt_r,nxt_f
        
        print(state_probabilities,i)
        
        alpha.append(dict(state_probabilities))

In [45]:
forward_algorithm(state_probabilities,alpha)

{'foggy': 0.10466842305689616, 'rainy': 0.01028273378316844, 'sunny': 0.7275295431588802} no
{'foggy': 0.11439244744052589, 'rainy': 0.014322403643966827, 'sunny': 0.5509569096977757} no
{'foggy': 0.09994659997395897, 'rainy': 0.013477988188229149, 'sunny': 0.425060076657645} no
{'foggy': 0.03541154127865861, 'rainy': 0.048892357690394996, 'sunny': 0.030717643357287067} yes
{'foggy': 0.023262672469707552, 'rainy': 0.007572411677204941, 'sunny': 0.03781612538208323} no
{'foggy': 0.013285361262055288, 'rainy': 0.002484709468915218, 'sunny': 0.033273556207167626} no
{'foggy': 0.003698874275561314, 'rainy': 0.005792066099421651, 'sunny': 0.002524451644677926} yes
{'foggy': 0.0010701947230657546, 'rainy': 0.003713873677663832, 'sunny': 0.0003322020065376995} yes
{'foggy': 0.0009907063866998222, 'rainy': 0.00046607426572577756, 'sunny': 0.001110035196587303} no
{'foggy': 0.00023261046767684336, 'rainy': 0.0005031104968301476, 'sunny': 0.00010023126381815604} yes


## Decoding Algorithm (Viterbi): Here we take the most probable state at each timestep to decode the sequence of hidden states

In [54]:
print(alpha)
def decode(alpha,predicted_sequence):
    for i in alpha:
        predicted_sequence.append(max(i,key=i.get))

[{'foggy': 0, 'rainy': 0, 'sunny': 1}, {'foggy': 0.10466842305689616, 'rainy': 0.01028273378316844, 'sunny': 0.7275295431588802}, {'foggy': 0.11439244744052589, 'rainy': 0.014322403643966827, 'sunny': 0.5509569096977757}, {'foggy': 0.09994659997395897, 'rainy': 0.013477988188229149, 'sunny': 0.425060076657645}, {'foggy': 0.03541154127865861, 'rainy': 0.048892357690394996, 'sunny': 0.030717643357287067}, {'foggy': 0.023262672469707552, 'rainy': 0.007572411677204941, 'sunny': 0.03781612538208323}, {'foggy': 0.013285361262055288, 'rainy': 0.002484709468915218, 'sunny': 0.033273556207167626}, {'foggy': 0.003698874275561314, 'rainy': 0.005792066099421651, 'sunny': 0.002524451644677926}, {'foggy': 0.0010701947230657546, 'rainy': 0.003713873677663832, 'sunny': 0.0003322020065376995}, {'foggy': 0.0009907063866998222, 'rainy': 0.00046607426572577756, 'sunny': 0.001110035196587303}, {'foggy': 0.00023261046767684336, 'rainy': 0.0005031104968301476, 'sunny': 0.00010023126381815604}]


In [55]:
predicted_sequence=[]
decode(alpha,predicted_sequence)

## Now let's print the predicted sequence:

In [59]:
print(predicted_sequence)
print("\n## Ignore the initial state since we already know it was sunny ##")

['sunny', 'sunny', 'sunny', 'sunny', 'rainy', 'sunny', 'sunny', 'rainy', 'rainy', 'sunny', 'rainy']

## Ignore the initial state since we already know it was sunny ##


## Now let's print the "a(ij)" - transition probability matrix:

In [62]:
tran_probFrame.T

,foggy,rainy,sunny
foggy,0.501916,0.291188,0.206897
rainy,0.224490,0.579592,0.195918
sunny,0.150101,0.054767,0.795132


## Now let's print the "b" - emission probability matrix:

In [63]:
prob_dframe

,foggy,rainy,sunny
no,0.697318,0.187755,0.91498
yes,0.302682,0.812245,0.08502


## Now let's print the "alpha" - at each timestep:

In [83]:
alpha_DataFrame=(pd.DataFrame(alpha))
index_arr=[]
index_arr.append("Initial Probability(Timestep-0)")
for i in range(1,len(alpha)-1):
    index_arr.append("Timestep - "+str(i))
index_arr.append("Final Timestep Probability(Used to calculate Wo)")
alpha_DataFrame.index=index_arr
print(alpha_DataFrame)

                                                     foggy     rainy     sunny
Initial Probability(Timestep-0)                   0.000000  0.000000  1.000000
Timestep - 1                                      0.104668  0.010283  0.727530
Timestep - 2                                      0.114392  0.014322  0.550957
Timestep - 3                                      0.099947  0.013478  0.425060
Timestep - 4                                      0.035412  0.048892  0.030718
Timestep - 5                                      0.023263  0.007572  0.037816
Timestep - 6                                      0.013285  0.002485  0.033274
Timestep - 7                                      0.003699  0.005792  0.002524
Timestep - 8                                      0.001070  0.003714  0.000332
Timestep - 9                                      0.000991  0.000466  0.001110
Final Timestep Probability(Used to calculate Wo)  0.000233  0.000503  0.000100


## Now let's calculate the final probability for our sequence 

### First calculate the probability of occurence of each state in our total dataset

In [84]:
total_sunny_p=(counts["sunny"]["yes"]+counts["sunny"]["no"])/len(data)
total_rainy_p=(counts["rainy"]["yes"]+counts["rainy"]["no"])/len(data)
total_foggy_p=(counts["foggy"]["yes"]+counts["foggy"]["no"])/len(data)

### Now we calculate the final probability for our sequence by transitioning to the Wo State

In [85]:
final_p=(total_sunny_p*alpha[-1]["sunny"])+(total_rainy_p*alpha[-1]["rainy"])+(total_foggy_p*alpha[-1]["foggy"])
print("Final Probability of our Predicted Sequence of Observation = ",final_p)

Final Probability of our Predicted Sequence of Observation =  0.00023348764811321137
